In [113]:
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [114]:
import qiskit_metal as metal 
from qiskit_metal import MetalGUI, open_docs, Dict, designs, draw 

In [115]:
design = designs.DesignPlanar()

In [116]:
gui = MetalGUI(design)

In [117]:
design.overwrite_enabled = True

In [118]:
design.chips.main.size.size_x = '5mm'
design.chips.main.size.size_y = '6mm'

In [119]:
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.qubits.transmon_cross_fl import TransmonCrossFL

In [120]:
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.pathfinder import RoutePathfinder 
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight

In [121]:
from qiskit_metal.qlibrary.lumped.cap_3_interdigital import Cap3Interdigital
from qiskit_metal.qlibrary.couplers.cap_n_interdigital_tee import CapNInterdigitalTee
from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond

In [122]:
Q_0 = TransmonPocket6( design, 'Q_0', options = dict(
    pos_x = '-0.75mm',
    pos_y = '0.0mm',
    gds_cell_name = 'FakeJunction_01',
    hfss_inductance = '14nH', 
    pad_width = '425um',
    pocket_height = '650um',
    connection_pads = dict( 
        readout = dict( loc_W = 0, loc_H = 1, pad_width = '80um', pad_gap = '50um'),
        bus_01 = dict( loc_W = 1, loc_H = -1, pad_width = '60um', pad_gap = '10um'),
        bus_02 = dict( loc_W = -1, loc_H = -1, pad_width = '60um', pad_gap = '10um'))))
gui.rebuild()
gui.autoscale()

In [123]:
Q_1 = TransmonPocket6( design, 'Q_1', options = dict(
    pos_x = '0.75mm',
    pos_y = '0.0mm',
    gds_cell_name = 'FakeJunction_01',
    hfss_inductance = '14nH', 
    pad_width = '425um',
    pocket_height = '650um',
    connection_pads = dict( 
        readout = dict( loc_W = 0, loc_H = 1, pad_width = '80um', pad_gap = '50um'),
        bus_12 = dict( loc_W = -1, loc_H = -1, pad_width = '60um', pad_gap = '10um'),
        bus_01 = dict( loc_W = 1, loc_H = -1, pad_width = '60um', pad_gap = '10um'))))
gui.rebuild()
gui.autoscale()

In [124]:
Q_2 = TransmonCrossFL( design, 'Q_2', options = dict(pos_x ='0.5mm', pos_y = '-1.5mm', orientation = '90',
                                                     connection_pads = dict(
                                                         bus_02 = dict( connector_location = '90', claw_length = '30um'),
                                                         bus_12 = dict( connector_location = '180', claw_length = '50um'),
                                                         readout = dict( connector_location = '0')),
                                                      fl_options = dict()))
gui.rebuild()
gui.autoscale()
                                    

In [125]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength 

In [126]:
def find_resonator_len( freq, line_width, line_gap, N):
    # freq(Hz), line_width(m), line_gap(m)
    # freq = frequancy in GHZ
    # line_width/line_gap in um
    # N --> 2 for lambda/2, 4 for lambda/4
    # subtrate dimensions and properties already set 
    [lambdaG, etfSqrt, q] = guided_wavelength( freq*10**9, line_width*10**-6, line_gap*10**-6, 750*10**-6, 200*10**-6, 11.9)
    # substrate_thickness = 750um
    # film_thickness = 200um
    # dielectric_constant = 11.9
    return str(lambdaG/N*10**3)+"mm"

In [127]:
?guided_wavelength

Signature:
guided_wavelength(
    freq,
    line_width,
    line_gap,
    substrate_thickness,
    film_thickness,
    dielectric_constant=11.45,
)
Docstring:
A simple calculator to determine the guided wavelength of a planar CPW
transmission line. Assumes the substrate has relative permeability of 1.
Assumes package grounds are far away.

Args:
    freq (float): The frequency of interest, in Hz (eg. 5*10**9).
    line_width (float): The width of the CPW trace (center) line, in meters (eg. 10*10**-6).
    line_gap (float): The width of the CPW gap (dielectric space), in meters (eg. 6*10**-6).
    substrate_thickness (float): Thickness of the dielectric substrate, in meters (eg. 760*10**-6).
    film_thickness (float): Thickness of the thin film, in meters (eg. 200*10**-9).
    dielectric_constant (float): The relative permittivity of the substrate.
        Defaults to 11.45, the value for Silicon at cryogenic temperatures.

Returns:
    tuple: Contents outlined below

Tuple contents:
 

In [128]:
bus_01 = RouteStraight( design, 'Bus_01', options = dict( 
                                                    hfss_wire_bond = True, 
                                                    pin_inputs = Dict( 
                                                        start_pin = Dict( component = 'Q_0', pin = 'bus_01'),
                                                        end_pin = Dict( component = 'Q_1', pin = 'bus_12')),
                                                    trace_width = '10um',
                                                    trace_gap = '6um'))
gui.rebuild()
gui.autoscale()

In [129]:
find_resonator_len( 5.8, 10, 6, 2)

'23.9145076345972mm'

In [130]:
bus_02 = RouteMeander( design, 'Bus_02', options = dict( pin_inputs = Dict( 
                                                                start_pin = Dict( component = 'Q_0', pin = 'bus_02'),
                                                                end_pin = Dict( component = 'Q_2', pin = 'bus_02')), 
                                                        lead = Dict( start_straight = '50um', end_straight = '125um'),
                                                        meander = Dict( asymmetry = '50um'),
                                                        fillet = '99um',
                                                        total_length = '10mm',
                                                       orientation = '-90'))
gui.rebuild()
gui.autoscale()

In [131]:
RouteMeander.get_template_options(design)

{'chip': 'main',
 'layer': '1',
 'pin_inputs': {'start_pin': {'component': '', 'pin': ''},
  'end_pin': {'component': '', 'pin': ''}},
 'fillet': '0',
 'lead': {'start_straight': '0mm',
  'end_straight': '0mm',
  'start_jogged_extension': '',
  'end_jogged_extension': ''},
 'total_length': '7mm',
 'trace_width': 'cpw_width',
 'meander': {'spacing': '200um', 'asymmetry': '0um'},
 'snap': 'true',
 'prevent_short_edges': 'true',
 'hfss_wire_bonds': False,
 'q3d_wire_bonds': False,
 'aedt_q3d_wire_bonds': False,
 'aedt_hfss_wire_bonds': False}

In [133]:
from collections import OrderedDict
jogs_start = OrderedDict()
# jogs_start[0] = ["L", '-400um']
# jogs_start[1] = ["L", '50um']


bus_12 = RouteMeander( design, 'Bus_12', options = dict( pin_inputs = Dict( 
                                                                start_pin = Dict( component = 'Q_1', pin = 'bus_01'),
                                                                end_pin = Dict( component = 'Q_2', pin = 'bus_12')), 
                                                        lead = Dict( start_straight = '125um', end_straight = '50um', 
                                                                    # start_jogged_extension = jogs_start
                                                                   ),
                                                        meander = Dict( asymmetry = '50um'),
                                                        fillet = '99um',
                                                        total_length = '10mm',
                                                       orientation = '-90'))
gui.rebuild()
gui.autoscale()

In [3]:
# Importing the necessary libraries
from qiskit_metal import designs, draw, MetalGUI
from qiskit_metal.qlibrary.core import QComponent
from qiskit_metal.qlibrary.components import Circle

# Create a new design
design = designs.DesignPlanar()

# Launch the GUI to visualize the design
gui = MetalGUI(design)

# Define the first Al patterned ring
class AlRing(QComponent):
    default_options = dict(
        inner_radius='50um',
        outer_radius='100um',
        layer='1'
    )

    def make(self):
        p = self.parse_options()
        
        # Draw the ring
        inner_circle = draw.Circle((0, 0), p.inner_radius)
        outer_circle = draw.Circle((0, 0), p.outer_radius)
        ring = outer_circle - inner_circle
        
        # Add to qgeometry
        self.add_qgeometry('poly', {'ring': ring}, layer=p.layer)

# Add the first ring to the design
ring1 = AlRing(design, 'ring1')

# Define the second Al patterned ring
class AlRing2(QComponent):
    default_options = dict(
        inner_radius='30um',
        outer_radius='80um',
        layer='1'
    )

    def make(self):
        p = self.parse_options()
        
        # Draw the ring
        inner_circle = draw.Circle((0, 0), p.inner_radius)
        outer_circle = draw.Circle((0, 0), p.outer_radius)
        ring = outer_circle - inner_circle
        
        # Add to qgeometry
        self.add_qgeometry('poly', {'ring': ring}, layer=p.layer)

# Add the second ring to the design
ring2 = AlRing2(design, 'ring2')

# Adjust positions
ring1.options.pos_x = 0
ring1.options.pos_y = 0

ring2.options.pos_x = 200
ring2.options.pos_y = 0

# Rebuild and visualize the design
design.rebuild()
gui.rebuild()
gui.autoscale()

# To display the GUI (uncomment the next line if running locally)
# gui.show()


ModuleNotFoundError: No module named 'qiskit_metal.qlibrary.components'

In [4]:
# Importing the necessary libraries
from qiskit_metal import designs, draw, MetalGUI
from qiskit_metal.qlibrary.core import QComponent
from qiskit_metal.qlibrary.components import TransmonPocket, Circle

# Create a new design
design = designs.DesignPlanar()

# Launch the GUI to visualize the design
gui = MetalGUI(design)

# Define the first aluminum ring
class AlRing(QComponent):
    default_options = dict(
        inner_radius='50um',
        outer_radius='100um',
        layer='1'
    )

    def make(self):
        p = self.parse_options()
        
        # Draw the ring
        inner_circle = draw.Circle((0, 0), p.inner_radius)
        outer_circle = draw.Circle((0, 0), p.outer_radius)
        ring = outer_circle - inner_circle
        
        # Add to qgeometry
        self.add_qgeometry('poly', {'ring': ring}, layer=p.layer)

# Add the first ring to the design
ring1 = AlRing(design, 'ring1')

# Define the second aluminum ring
class AlRing2(QComponent):
    default_options = dict(
        inner_radius='30um',
        outer_radius='80um',
        layer='1'
    )

    def make(self):
        p = self.parse_options()
        
        # Draw the ring
        inner_circle = draw.Circle((0, 0), p.inner_radius)
        outer_circle = draw.Circle((0, 0), p.outer_radius)
        ring = outer_circle - inner_circle
        
        # Add to qgeometry
        self.add_qgeometry('poly', {'ring': ring}, layer=p.layer)

# Add the second ring to the design
ring2 = AlRing2(design, 'ring2')

# Define the superconducting qubit
qubit = TransmonPocket(design, 'Qubit',
    options=dict(
        pad_width='425um',
        pad_height='90um',
        pad_gap='20um',
        connection_pads=dict(
            a=dict(loc_W=+1, loc_H=0),
            b=dict(loc_W=-1, loc_H=0)
        )
    )
)

# Adjust positions
ring1.options.pos_x = 0
ring1.options.pos_y = 0

ring2.options.pos_x = 300
ring2.options.pos_y = 0

qubit.options.pos_x = 150
qubit.options.pos_y = 0

# Rebuild and visualize the design
design.rebuild()
gui.rebuild()
gui.autoscale()

# To display the GUI (uncomment the next line if running locally)
# gui.show()


ModuleNotFoundError: No module named 'qiskit_metal.qlibrary.components'

In [5]:
# Importing the necessary libraries
from qiskit_metal import designs, draw, MetalGUI
from qiskit_metal.qlibrary.core import QComponent
# from qiskit_metal.qlibrary.components import Circle, TransmonPocket

# Create a new design
design = designs.DesignPlanar()

# Launch the GUI to visualize the design
gui = MetalGUI(design)

# Define the sapphire substrate
class SapphireSubstrate(QComponent):
    default_options = dict(
        width='15.5mm',
        height='25.4mm',
        layer='0'
    )

    def make(self):
        p = self.parse_options()
        
        # Draw the substrate
        substrate = draw.rectangle(p.width, p.height)
        
        # Add to qgeometry
        self.add_qgeometry('poly', {'substrate': substrate}, layer=p.layer)

# Add the substrate to the design
substrate = SapphireSubstrate(design, 'Sapphire_Substrate')

# Define the first aluminum layer of the WGM resonator
class WGMLayer1(QComponent):
    default_options = dict(
        inner_radius='3mm',
        outer_radius='4mm',
        layer='1'
    )

    def make(self):
        p = self.parse_options()
        
        # Draw the ring for the first layer
        inner_circle = draw.Circle((0, 0), p.inner_radius)
        outer_circle = draw.Circle((0, 0), p.outer_radius)
        ring = outer_circle - inner_circle
        
        # Add to qgeometry
        self.add_qgeometry('poly', {'ring': ring}, layer=p.layer)

# Add the first WGM layer to the design
wgm_layer1 = WGMLayer1(design, 'WGM_Layer1')

# Define the second aluminum layer of the WGM resonator
class WGMLayer2(QComponent):
    default_options = dict(
        inner_radius='3.1mm',
        outer_radius='4.1mm',
        layer='2'
    )

    def make(self):
        p = self.parse_options()
        
        # Draw the ring for the second layer
        inner_circle = draw.Circle((0, 0), p.inner_radius)
        outer_circle = draw.Circle((0, 0), p.outer_radius)
        ring = outer_circle - inner_circle
        
        # Add to qgeometry
        self.add_qgeometry('poly', {'ring': ring}, layer=p.layer)

# Add the second WGM layer to the design
wgm_layer2 = WGMLayer2(design, 'WGM_Layer2')

# Define the superconducting qubit
qubit = TransmonPocket(design, 'Qubit',
    options=dict(
        pad_width='425um',
        pad_height='90um',
        pad_gap='20um',
        connection_pads=dict(
            a=dict(loc_W=+1, loc_H=0),
            b=dict(loc_W=-1, loc_H=0)
        ),
        layer='1'
    )
)

# Adjust positions to match fabrication details
substrate.options.pos_x = 0
substrate.options.pos_y = 0

wgm_layer1.options.pos_x = 0
wgm_layer1.options.pos_y = 0

wgm_layer2.options.pos_x = 0
wgm_layer2.options.pos_y = 0

qubit.options.pos_x = -2.5  # Position relative to WGM resonator center
qubit.options.pos_y = 0

# Set thickness and material properties
# Note: Qiskit Metal doesn't directly support material properties yet,
# but we set them here conceptually

wgm_layer1.options.thickness = '20nm'
wgm_layer2.options.thickness = '30nm'

# Rebuild and visualize the design
design.rebuild()
gui.rebuild()
gui.autoscale()

# To display the GUI (uncomment the next line if running locally)
# gui.show()


ModuleNotFoundError: No module named 'qiskit_metal.qlibrary.components'

In [7]:
# Importing the necessary libraries
from qiskit_metal import designs, draw, MetalGUI
from qiskit_metal.qlibrary.core import QComponent
# from qiskit_metal.qlibrary.components import Circle, TransmonPocket

# Create a new design
design = designs.DesignPlanar()

# Launch the GUI to visualize the design
gui = MetalGUI(design)

# Define the sapphire substrate
class SapphireSubstrate(QComponent):
    default_options = dict(
        width='15.5mm',
        height='25.4mm',
        layer='0'
    )

    def make(self):
        p = self.parse_options()
        
        # Draw the substrate
        substrate = draw.rectangle(p.width, p.height)
        
        # Add to qgeometry
        self.add_qgeometry('poly', {'substrate': substrate}, layer=p.layer)

# Add the substrate to the design
substrate = SapphireSubstrate(design, 'Sapphire_Substrate')
gui.rebuild()
gui.autoscale()